In [58]:
import pandas as pd
import numpy as np
import urllib.request as req
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
req.urlretrieve(url,"mushdata.csv")

('mushdata.csv', <http.client.HTTPMessage at 0x21d0d5a9710>)

In [59]:
mr=pd.read_csv("data/agaricus-lepiota.data", header=None)
mr.shape

(8124, 23)

In [60]:
# 데이터를 숫자로 변환
#print(ord('a')) # 아스키코드 출력
#print(chr(97))
# 문자를 숫자로 변환할 때, ord 함수를 적절히 활용하고자 함
from sklearn.model_selection import train_test_split # train, test data 분리

label=[]
data=[]
for row_index, row in mr.iterrows(): #데이터 프레임의 행을 반복하며 행 자체를 포함하는 객체에 덧붙여 각 행의 색인을 반환하는 제너레이터
    label.append(row.ix[0]) # 독버섯인지 아닌지를 판단하는 y 레이블 추출
    row_data=[]
    for v in row.ix[1:]:
        row_data.append(ord(v)) #ascii:a-97,b-98,... , 한행에 해당하는 데이터가 아스키 코드로 변환되어 저장되어 리스트에 저장
    data.append(row_data) # 모든 행을 담기 위해 append 해줌

# label # y레이블이 저장된 것을 확인할 수 있음
# data # 아스키 코드로 변환되어 저장된 것을 확인 할 수 있음

data_train,data_test, label_train, label_test=train_test_split(data,label)
# data와 label도 분리해줌

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [61]:
#(8124, 23)
len(data_train) #6093
len(data_test) #2031
# 데이터를 train과 test를 알아서 나눠줌

2031

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [63]:
# 데이터 학습 => 모델 생성
clf=RandomForestClassifier()
clf.fit(data_train, label_train) 

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [64]:
predict=clf.predict(data_test)
predict
# 예측 : array(['e', 'e', 'e', ..., 'e', 'e', 'p'], dtype='<U1')
# 실제 : label_test 변수에 저장되어있는 값
# 실제와 예측을 비교해서 얼마나 정확하게 예측되었는지 확인

array(['e', 'p', 'e', ..., 'e', 'e', 'e'], dtype='<U1')

In [65]:
# 실제와 예측간 정확도
as_score=metrics.accuracy_score(label_test, predict)
as_score

1.0

In [66]:
# 정확도를 확인할 수 있는 여러 방법
rep=metrics.classification_report(label_test, predict) 
# 정밀도(precision), 재현율(recall), F1점수(f1-score) 출력하는 함수, support는 데이터 개수
print(rep) 

              precision    recall  f1-score   support

           e       1.00      1.00      1.00      1031
           p       1.00      1.00      1.00      1000

   micro avg       1.00      1.00      1.00      2031
   macro avg       1.00      1.00      1.00      2031
weighted avg       1.00      1.00      1.00      2031



빨강 = 1, 파랑 = 2, 초록 = 3, 흰색 =4  
빨강 = 1 => 1000  
파랑 = 2 => 0100  
....  
위와 같이 수치값이 분류를 하기 위한 변수인 경우 원핫인코딩으로 나타내야함  
하지만 온도처럼 의미가 있는 연속 변수인 경우 연속된 값으로 나타내면 됨 

In [67]:
label=[]
new_data=[]
attr_list=[]

In [68]:
# train, test data 분리를 직접 쾨딩해보기
for row_index, row in mr.iterrows():
    label.append(row.ix[0])
    exdata=[]
    for col,v in enumerate(row.ix[1:]): # 열거형 형태(해당 번호와 값)로 출력, col:인덱스(0부터 시작)와 v:해당 값 출력
        #print(v)
        if row_index==0:
            attr={"dic":{}, "cnt":0} # 딕셔너리 안에 딕셔너리가 중첩되어있는 구조
            # attr={"dic":{    }, "cnt" : 0 } => 딕셔너리 초기화 작업 
            attr_list.append(attr)
            # attr_list=[ attr={"dic":{    }, "cnt" : 0 } ]
        else : 
            attr=attr_list[col]
        d=[0,0,0,0,0,0,0,0,0,0,0,0]
        if v in attr["dic"] : # 비어있는는 딕셔너리와 리스트를 false로 출력 
            idx=attr["dic"][v]
        else:
            idx=attr["cnt"] # 0출력
            attr["dic"][v]=idx
            attr["cnt"] += 1
        d[idx] =1
        exdata += d #2차원
    new_data.append(exdata)
# 버섯의 특징을 표현하는 변수의 종류가 최대 12개라고 전체        
# x1 (모양 3가지) : [0,0,0]
# x2 (색상 5가지) : [0,0,0,0,0]
# ...
# [0,0,0,0,0,0,0,0]

# train, test 분리 함수 사용
data_train,data_test, label_train, label_test=train_test_split(data,label)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
